In [ ]:
%run ./00_Mapping_Setup

In [ ]:
/* ===================================================================================
   METRIC: EMP01 - Canadian Officer / PEP Presence
   
   BUSINESS QUESTION: 
   At anytime during the assessment period, did the unit hire or promote any POs...
   
   LOGIC SUMMARY:
   1. Filters the PEP List for Region = 'Canada'.
   2. Cleans the Cost Center string by grabbing the ID before the first space.
   3. Joins the Canadian officers to the standardized CC Mapping Bootstrap view.
   4. Rolls up the data by AU_ID (BusinessID). If 1 or more Canadian officers 
      are linked to that AU, the Response outputs 'Yes'. Otherwise, 'No'.
=================================================================================== */

WITH Canadian_PEPs AS (
    -- 1. Grab all valid Cost Center IDs from Canadian employees
    SELECT DISTINCT
        -- Cleans the Costcenter: grabs the string before the first space
        TRIM(SUBSTRING_INDEX(Costcenter, ' ', 1)) AS Cleaned_CC
    FROM hive_metastore.ra_adido_2025.employee_pep_list_as_of_oct312025
    WHERE TRIM(Region) = 'Canada'
      AND Costcenter IS NOT NULL
),

Mapped_AUs AS (
    -- 2. Join the base AU list to the PEP matches
    SELECT 
        m.AU_ID AS BusinessID,
        -- Flag if the Cost Center successfully matched a Canadian PEP
        CASE WHEN p.Cleaned_CC IS NOT NULL THEN 1 ELSE 0 END AS Has_PEP
    FROM vw_cost_center_mapping_bootstrap m
    
    LEFT JOIN Canadian_PEPs p
        -- Smart Join: Adds a leading '0' only if the Mapping ID is exactly 3 characters
        ON CASE 
            WHEN LENGTH(TRIM(CAST(m.Cost_Center_ID AS STRING))) = 3 
            THEN CONCAT('0', TRIM(CAST(m.Cost_Center_ID AS STRING))) 
            ELSE TRIM(CAST(m.Cost_Center_ID AS STRING)) 
           END = p.Cleaned_CC
)

-- 3. Roll everything up to match the Final Output Template
SELECT 
    BusinessID,                          
    'EMP01' AS QuestionID,               
    
    -- Maps to Column E (Response)
    CASE 
        WHEN SUM(Has_PEP) > 0 THEN 'Yes' 
        ELSE 'No' 
    END AS Response
    
FROM Mapped_AUs
GROUP BY BusinessID
ORDER BY BusinessID;